## Setup

In [1]:
%cd ..

d:\Blockchain\data\orai\orchai_validator


In [2]:
from labeling.etl_processor import ETLProcessor
from labeling.utils import get_spark, query, upload
from omegaconf import OmegaConf
import pyspark.sql.functions as F

## ETL

In [3]:
spark = get_spark()

def sample(spark):
    config = OmegaConf.load("./config/local.yaml")
    df = query(spark, **config['from'])
    df.printSchema()
    max_height = df.agg(F.max(df.block_height)).collect()[0].asDict()['max(block_height)']
    df = df.filter(df.block_height < max_height)
    return df

In [4]:
df = sample(spark)

Successfully queried data from database
root
 |-- block_height: integer (nullable = true)
 |-- operator_address: string (nullable = true)
 |-- jailed: boolean (nullable = true)
 |-- status: string (nullable = true)
 |-- tokens: decimal(38,0) (nullable = true)
 |-- commission_rate: float (nullable = true)
 |-- delegator_shares: decimal(38,0) (nullable = true)
 |-- self_bonded: decimal(38,0) (nullable = true)
 |-- propose: boolean (nullable = true)
 |-- vote: boolean (nullable = true)



In [7]:
accept_rate = 0.1
concentration_level = 0.9
vote_score = 2
propose_score = 6
window_size = 600
A = 9
B = 4
C = 2
D = 4

df = ETLProcessor.data_scoring(df, accept_rate, concentration_level, vote_score, propose_score, window_size, A, B, C, D)

------------------------------------------------
Successfully converted voting_power_score column
------------------------------------------------
------------------------------------------------
Successfully converted commission_score column
------------------------------------------------
------------------------------------------------
Successfully converted self_bonded_score column
------------------------------------------------
------------------------------------------------
Successfully converted vote_propose_score column
------------------------------------------------
------------------------------------------------
Sucessfully converted final_score
------------------------------------------------


In [7]:
# df.write.csv("./data/orchai_etl", header=True)

## Upload

In [38]:
df = df.drop('jailed', 'status', 'tokens', 'commission_rate', 'delegator_shares', 'self_bonded', 'propose', 'vote')
config = OmegaConf.load("./config/local.yml")
upload(df, **config.to)